In [7]:
# ═══════════════════════════════════════════════════════════
# 🔬 Skin Cancer - ResNet50 (성공 방법!)
# 전체 학습 + 10 epochs + 높은 LR
# ═══════════════════════════════════════════════════════════

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import gc

print("=" * 60, flush=True)
print("🔬 Skin Cancer - ResNet50 학습", flush=True)
print("=" * 60, flush=True)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 설정 (성공한 방법 기반!)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

IMG_SIZE = 224
BATCH_SIZE = 32
LEARNING_RATE = 0.001  # 0.0005 → 0.001 (2배!)
EPOCHS = 10  # 30 → 10 (빠른 수렴!)
SEED = 42

print(f"\n📋 설정:", flush=True)
print(f"  • 모델: ResNet50 (EfficientNet 아님!)", flush=True)
print(f"  • 학습 방식: 전체 학습 (Freeze 없음!)", flush=True)
print(f"  • Learning Rate: {LEARNING_RATE} (높음!)", flush=True)
print(f"  • Epochs: {EPOCHS} (짧음!)", flush=True)
print(f"  • 목표: 80-85%+", flush=True)

# Seed 설정
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 1. 데이터 로드 (기존 전처리 파일 사용!)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n📂 데이터 로드 중...", flush=True)

DATA_PATH = '/kaggle/working/skin_cancer_data_IMG224.npz'

data = np.load(DATA_PATH)
X_data = data['X']
y_data = data['y']

print("✅ 데이터 로드 완료!", flush=True)
print(f"  • X shape: {X_data.shape}", flush=True)
print(f"  • Benign: {np.sum(y_data==0):,}개", flush=True)
print(f"  • Malignant: {np.sum(y_data==1):,}개", flush=True)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 2. Train/Val 분할
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n✂️ Train/Val 분할 중...", flush=True)

X_train_uint8, X_val_uint8, y_train, y_val = train_test_split(
    X_data, y_data,
    test_size=0.2,
    random_state=SEED,
    stratify=y_data
)

print("✅ 분할 완료!", flush=True)
print(f"  • Train: {len(X_train_uint8):,}개", flush=True)
print(f"  • Val: {len(X_val_uint8):,}개", flush=True)

del X_data, y_data, data
gc.collect()

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 3. ImageNet 정규화
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

MEAN = np.array([123.675, 116.28, 103.53], dtype=np.float32)
STD = np.array([58.395, 57.12, 57.375], dtype=np.float32)

def normalize_imagenet(X):
    """ImageNet 정규화 - ResNet50은 이걸 기대함"""
    X = X.astype(np.float32)
    for i in range(3):
        X[:, :, :, i] = (X[:, :, :, i] - MEAN[i]) / STD[i]
    return X

print("\n🔄 정규화 중...", flush=True)

X_train = normalize_imagenet(X_train_uint8)
del X_train_uint8
gc.collect()

X_val = normalize_imagenet(X_val_uint8)
del X_val_uint8
gc.collect()

print("✅ 정규화 완료!", flush=True)
print(f"  • Train range: [{X_train.min():.4f}, {X_train.max():.4f}]", flush=True)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 4. ResNet50 모델 구축 (핵심!)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n🏗️ ResNet50 모델 구축 중...", flush=True)

# ResNet50 Base Model
base_model = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling=None
)

# ⭐ 핵심 차이: 처음부터 전체 학습!
base_model.trainable = True
print("  ⭐ Base model: TRAINABLE (Freeze 없음!)", flush=True)

# Classification Head (간단하게!)
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=True)  # training=True 명시
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

print("✅ ResNet50 모델 구축 완료!", flush=True)
print(f"  • Total params: {model.count_params():,}", flush=True)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 5. 컴파일 (높은 LR!)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

METRICS = [
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.AUC(name='auc'),
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=METRICS
)

print("✅ 컴파일 완료!", flush=True)
print(f"  • Optimizer: Adam(lr={LEARNING_RATE})", flush=True)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 6. Data Augmentation (더 강력하게!)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n🔄 Augmentation Generator 생성 중...", flush=True)

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,  # 피부암은 상하 반전도 OK
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

train_generator = train_datagen.flow(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

print(f"✅ Generator 생성 완료!", flush=True)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 7. Callbacks
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

callbacks = [
    ModelCheckpoint(
        'best_model_resnet50.keras',
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=5,  # 10 epochs 중 5번
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-6,
        verbose=1
    )
]

print("✅ Callbacks 준비 완료!", flush=True)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 8. 학습 시작! (한 번에!)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 60, flush=True)
print("🔥 ResNet50 학습 시작! (10 Epochs)", flush=True)
print("=" * 60, flush=True)
print("\n⏰ 예상 시간: 약 30-40분 (짧음!)", flush=True)
print("💡 Freeze 없이 바로 전체 학습!", flush=True)
print("⚠️ 세션 유지: 클릭 유지!\n", flush=True)

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

print("\n✅ 학습 완료!", flush=True)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 9. 최종 평가
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 60, flush=True)
print("📈 최종 평가", flush=True)
print("=" * 60, flush=True)

# Best 모델 로드
best_model = keras.models.load_model('best_model_resnet50.keras')

# 최고 성능 epoch 찾기
best_acc = max(history.history['val_accuracy'])
best_epoch = history.history['val_accuracy'].index(best_acc) + 1
best_recall = history.history['val_recall'][best_epoch - 1]
best_precision = history.history['val_precision'][best_epoch - 1]

print(f"\n📊 Best Performance:", flush=True)
print(f"  • Epoch: {best_epoch}/{EPOCHS}", flush=True)
print(f"  • Accuracy: {best_acc*100:.2f}%", flush=True)
print(f"  • Recall: {best_recall*100:.2f}%", flush=True)
print(f"  • Precision: {best_precision*100:.2f}%", flush=True)

# 최종 평가
results = best_model.evaluate(X_val, y_val, verbose=0)

print(f"\n🔬 최종 검증 성능:", flush=True)
print(f"  • Loss: {results[0]:.4f}", flush=True)
print(f"  • Accuracy: {results[1]*100:.2f}%", flush=True)
print(f"  • Recall: {results[2]*100:.2f}%", flush=True)
print(f"  • Precision: {results[3]*100:.2f}%", flush=True)
print(f"  • AUC: {results[4]:.4f}", flush=True)

# 목표 달성 확인
print("\n🎯 목표 달성 체크:", flush=True)
print(f"  • Accuracy ≥ 70%: {'✅' if results[1] >= 0.70 else '❌'} ({results[1]*100:.2f}%)", flush=True)
print(f"  • Accuracy ≥ 80%: {'✅' if results[1] >= 0.80 else '❌'} ({results[1]*100:.2f}%)", flush=True)
print(f"  • Accuracy ≥ 85%: {'✅' if results[1] >= 0.85 else '❌'} ({results[1]*100:.2f}%)", flush=True)

if results[1] >= 0.85:
    print("\n🎉🎉 목표 초과 달성! (85%+) 🎉🎉", flush=True)
elif results[1] >= 0.80:
    print("\n🎉 훌륭합니다! (80%+)", flush=True)
elif results[1] >= 0.70:
    print("\n✅ 목표 달성! (70%+)", flush=True)
else:
    print(f"\n⚠️ 목표 미달... 현재 {results[1]*100:.2f}%", flush=True)

# 최종 모델 저장
best_model.save('final_model_resnet50.keras')
print(f"\n✅ 최종 모델 저장: final_model_resnet50.keras", flush=True)

print("\n" + "=" * 60, flush=True)
print("🎉 ResNet50 학습 완료!", flush=True)
print("=" * 60, flush=True)
print("\n💡 핵심 차이:", flush=True)
print("  • ResNet50 (EfficientNet 아님)", flush=True)
print("  • 전체 학습 (Freeze 없음)", flush=True)
print("  • 10 epochs (빠른 수렴)", flush=True)
print("  • LR 0.001 (높음)", flush=True)

🔬 Skin Cancer - ResNet50 학습

📋 설정:
  • 모델: ResNet50 (EfficientNet 아님!)
  • 학습 방식: 전체 학습 (Freeze 없음!)
  • Learning Rate: 0.001 (높음!)
  • Epochs: 10 (짧음!)
  • 목표: 80-85%+

📂 데이터 로드 중...
✅ 데이터 로드 완료!
  • X shape: (3908, 224, 224, 3)
  • Benign: 1,954개
  • Malignant: 1,954개

✂️ Train/Val 분할 중...
✅ 분할 완료!
  • Train: 3,126개
  • Val: 782개

🔄 정규화 중...
✅ 정규화 완료!
  • Train range: [-2.1179, 2.6400]

🏗️ ResNet50 모델 구축 중...
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
  ⭐ Base model: TRAINABLE (Freeze 없음!)
✅ ResNet50 모델 구축 완료!
  • Total params: 24,145,281
✅ 컴파일 완료!
  • Optimizer: Adam(lr=0.001)

🔄 Augmentation Generator 생성 중...
✅ Generator 생성 완료!
✅ Callbacks 준비 완료!

🔥 ResNet50 학습 시작! (10 Epochs)

⏰ 예상 시간: 약 30-40분 (짧음!)
💡 Freeze 없이 바로 전체 학습!
⚠️ 세션 유지: 클릭 유지!



/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 621ms/step - accuracy: 0.6534 - auc: 0.7082 - loss: 0.7195 - precision: 0.6319 - recall: 0.7546
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to best_model_resnet50.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 131s 740ms/step - accuracy: 0.6538 - auc: 0.7086 - loss: 0.7188 - precision: 0.6320 - recall: 0.7557 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 122454.8359 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step - accuracy: 0.6895 - auc: 0.7759 - loss: 0.6307 - precision: 0.6292 - recall: 0.9199
Epoch 2: val_accuracy did not improve from 0.50000
98/98 ━━━━━━━━━━━━━━━━━━━━ 40s 410ms/step - accuracy: 0.6896 - auc: 0.7759 - loss: 0.6306 - precision: 0.6293 - recall: 0.9199 - val_accuracy: 0.5000 - val_auc: 0.3066 - val_loss: 1.6211 - val_precision: 0.5000 - val_recall: 1.0000 - learning_rate: 0.0010
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 394ms/step - 

In [8]:
# ═══════════════════════════════════════════════════════════
# 📦 Streamlit용 파일 저장
# 모델 + 샘플 이미지 4장 (양성 2, 악성 2)
# ═══════════════════════════════════════════════════════════

import pandas as pd
import cv2
import os
import shutil
from pathlib import Path

print("=" * 60)
print("📦 Streamlit용 파일 저장")
print("=" * 60)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 1. 모델 파일 복사
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n📥 모델 파일 확인 중...")

model_path = 'final_model_resnet50.keras'

if os.path.exists(model_path):
    print(f"✅ 모델 존재: {model_path}")
    file_size = os.path.getsize(model_path) / 1024**2
    print(f"  • 크기: {file_size:.1f}MB")
else:
    print(f"❌ 모델 파일 없음!")
    print("  • 학습 코드를 먼저 실행하세요!")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 2. 샘플 이미지 저장 폴더 생성
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n📁 샘플 폴더 생성 중...")

sample_dir = 'streamlit_samples'
os.makedirs(sample_dir, exist_ok=True)

print(f"✅ 폴더 생성: {sample_dir}/")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 3. 메타데이터 로드
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n📊 메타데이터 로드 중...")

BASE_DIR = '/kaggle/input/skin-cancer-mnist-ham10000'
METADATA_PATH = f'{BASE_DIR}/HAM10000_metadata.csv'
IMAGE_DIR_1 = f'{BASE_DIR}/HAM10000_images_part_1'
IMAGE_DIR_2 = f'{BASE_DIR}/HAM10000_images_part_2'

df = pd.read_csv(METADATA_PATH)

# Binary 라벨 생성
MALIGNANT_CLASSES = ['mel', 'bcc', 'akiec']
df['binary_label'] = df['dx'].apply(lambda x: 1 if x in MALIGNANT_CLASSES else 0)

print(f"✅ 메타데이터 로드 완료!")
print(f"  • Benign (0): {(df['binary_label']==0).sum():,}개")
print(f"  • Malignant (1): {(df['binary_label']==1).sum():,}개")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 4. 샘플 이미지 선택 (양성 2, 악성 2)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n🎲 샘플 이미지 선택 중...")

# 양성 2장
benign_samples = df[df['binary_label'] == 0].sample(n=2, random_state=42)

# 악성 2장
malignant_samples = df[df['binary_label'] == 1].sample(n=2, random_state=42)

samples = pd.concat([benign_samples, malignant_samples])

print("✅ 샘플 선택 완료!")
print(f"\n선택된 샘플:")
for idx, row in samples.iterrows():
    label_str = "Benign" if row['binary_label'] == 0 else "Malignant"
    print(f"  • {row['image_id']}: {row['dx']} → {label_str}")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 5. 이미지 파일 찾기 및 복사
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def find_image_path(image_id):
    """2개 폴더에서 이미지 찾기"""
    filename = f"{image_id}.jpg"
    
    # 폴더 1 확인
    path1 = os.path.join(IMAGE_DIR_1, filename)
    if os.path.exists(path1):
        return path1
    
    # 폴더 2 확인
    path2 = os.path.join(IMAGE_DIR_2, filename)
    if os.path.exists(path2):
        return path2
    
    return None

print("\n📸 이미지 복사 중...")

for idx, (i, row) in enumerate(samples.iterrows(), 1):
    # 원본 경로 찾기
    src_path = find_image_path(row['image_id'])
    
    if src_path is None:
        print(f"  ❌ {idx}. {row['image_id']}: 파일 없음!")
        continue
    
    # 새 파일명
    label_str = "benign" if row['binary_label'] == 0 else "malignant"
    new_filename = f"sample_{idx}_{label_str}_{row['dx']}.jpg"
    dst_path = os.path.join(sample_dir, new_filename)
    
    # 복사
    shutil.copy(src_path, dst_path)
    
    print(f"  ✅ {idx}. {new_filename}")

print("\n✅ 이미지 복사 완료!")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 6. 샘플 메타데이터 저장
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n💾 샘플 메타데이터 저장 중...")

# 저장할 정보만 추출
samples_info = samples[['image_id', 'dx', 'binary_label']].copy()
samples_info['label_name'] = samples_info['binary_label'].apply(
    lambda x: 'Malignant' if x == 1 else 'Benign'
)

# CSV 저장
metadata_path = os.path.join(sample_dir, 'samples_info.csv')
samples_info.to_csv(metadata_path, index=False)

print(f"✅ 메타데이터 저장: {metadata_path}")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 7. 최종 확인
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 60)
print("📦 저장 완료!")
print("=" * 60)

print(f"\n✅ 모델 파일:")
print(f"  • {model_path}")

print(f"\n✅ 샘플 이미지 폴더:")
print(f"  • {sample_dir}/")

print(f"\n📥 다운로드할 파일:")
print(f"  1. {model_path}")
print(f"  2. {sample_dir}/ (폴더 전체)")

print("\n💡 Streamlit용:")
print(f"  • 모델: final_model_resnet50.keras")
print(f"  • 샘플 4장: streamlit_samples/ 폴더")
print(f"  • 메타정보: streamlit_samples/samples_info.csv")

print("\n🎯 다음 단계:")
print(f"  1. 위 파일들 다운로드")
print(f"  2. Streamlit 코드 작성")
print(f"  3. 로컬에서 실행!")

print("\n" + "=" * 60)

📦 Streamlit용 파일 저장

📥 모델 파일 확인 중...
✅ 모델 존재: final_model_resnet50.keras
  • 크기: 276.7MB

📁 샘플 폴더 생성 중...
✅ 폴더 생성: streamlit_samples/

📊 메타데이터 로드 중...
✅ 메타데이터 로드 완료!
  • Benign (0): 8,061개
  • Malignant (1): 1,954개

🎲 샘플 이미지 선택 중...
✅ 샘플 선택 완료!

선택된 샘플:
  • ISIC_0027239: bkl → Benign
  • ISIC_0032166: nv → Benign
  • ISIC_0031721: bcc → Malignant
  • ISIC_0029811: akiec → Malignant

📸 이미지 복사 중...
  ✅ 1. sample_1_benign_bkl.jpg
  ✅ 2. sample_2_benign_nv.jpg
  ✅ 3. sample_3_malignant_bcc.jpg
  ✅ 4. sample_4_malignant_akiec.jpg

✅ 이미지 복사 완료!

💾 샘플 메타데이터 저장 중...
✅ 메타데이터 저장: streamlit_samples/samples_info.csv

📦 저장 완료!

✅ 모델 파일:
  • final_model_resnet50.keras

✅ 샘플 이미지 폴더:
  • streamlit_samples/

📥 다운로드할 파일:
  1. final_model_resnet50.keras
  2. streamlit_samples/ (폴더 전체)

💡 Streamlit용:
  • 모델: final_model_resnet50.keras
  • 샘플 4장: streamlit_samples/ 폴더
  • 메타정보: streamlit_samples/samples_info.csv

🎯 다음 단계:
  1. 위 파일들 다운로드
  2. Streamlit 코드 작성
  3. 로컬에서 실행!

